# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2017, 9, 1, 10, 32, 51, 126028, tzinfo=datetime.timezone.utc), bid=1.1913, bidSize=17500000, ask=1.19135, askSize=1000000, lastSize=0, prevBid=1.19135, prevBidSize=17000000, prevAsk=1.1914, prevAskSize=18000000, high=1.193, low=1.1879, close=1.1909)

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.191325

The following cell will start a 15 second loop that prints a live updated ticker table.
It is updated with every new network packet that arrives.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]

for _ in ib.loopUntil(timeout=15):
    for i, t in enumerate(ib.tickers()):
        df.iloc[i, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
    clear_output(wait=True)
    display(df)

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,2000000,1.19145,1.1915,14500000,1.193,1.1879,1.1909
1,USDJPY,12000000,110.125,110.13,2000000,110.22,109.92,109.98
2,GBPUSD,1000000,1.29245,1.2925,7000000,1.29475,1.29055,1.2931
3,USDCHF,9000000,0.95965,0.9598,13500000,0.9615,0.9579,0.9587
4,USDCAD,16000000,1.24785,1.24795,4000000,1.2492,1.24505,1.2483
5,AUDUSD,2000000,0.7928,0.7929,17500000,0.79565,0.79215,0.79465


After an update the tick data is available in the 'ticks' attribute of the ticker.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

In [8]:
ib.disconnect()